In [27]:
from ortools.linear_solver import pywraplp
import numpy as np

In [38]:
def transportationProblem(M,S,D,min=True):
    lenS= len(S)
    lenD= len(D)
    solver = pywraplp.Solver.CreateSolver('GLOP')
    V = [
        [solver.IntVar(0, solver.infinity(),'x{0}{1}'.format(i,j))
             for j in range(lenD)] 
                for i in range(lenS)
    ]
    Z = sum(M[i][j]*V[i][j] for i in range(lenS) for j in range(lenD))
    for i in range(lenS):
        solver.Add(
            sum(M[i][j]*V[i][j] for j in range(lenD)) <= S[i]) 
    for j in range(lenD):
        solver.Add(
            sum(M[i][j]*V[i][j] for i in range(lenS)) >= D[j])
    
    if(min):
        solver.Minimize(Z)
    else:
        solver.Maximize(Z)
    status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('Objective value =', solver.Objective().Value())
        print(['x{0}{1} = {2}'.format(i,j,V[i][j].solution_value()) for i in range(lenS) for j in range(lenD)])
    else:
        print('The problem does not have an optimal solution.')

In [39]:
M = np.loadtxt('data.txt')
S = np.loadtxt('dataS.txt')
D = np.loadtxt('dataD.txt')

transportationProblem(M,S,D)

TypeError: unsupported operand type(s) for %: 'Variable' and 'int'